In [ ]:
import sys
sys.path.insert(0, '..')
import bug_lib as BL


# bug_version_list = [
#     [],
#     [5],
#     [6],
#     [7],
#     [8],
#     [10],
#     [11],
#     [12],
#     [13],
#     [14],
#     [15]
# ]

bug_version = []

BL.cover_then_inject_bugs(bug_version)

After Injecting bugs, Restart the ipy kernal.

In [ ]:
from stable_baselines3 import PPO, DQN, A2C
import gymnasium as gym
import imageio
import numpy as np
import os

def generate_gif(model, output_path, num_steps=350, fps=29):
    images = []
    obs = model.env.reset()
    img = model.env.render(mode="rgb_array")
    for i in range(num_steps):
        images.append(img)
        action, _ = model.predict(obs)
        obs, _, _, _ = model.env.step(action)
        img = model.env.render(mode="rgb_array")

    imageio.mimsave(output_path, [np.array(img) for i, img in enumerate(images) if i % 2 == 0], fps=fps)

def train_frozen_lake(testbed='Frozenlake', model_type='ppo', total_timesteps=3000, tb_log_name="run"):
    if testbed == 'Frozenlake':
        env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=False, render_mode="rgb_array")
    else:
        env = gym.make("MountainCarContinuous-v0", render_mode='rgb_array')

    # 检查输出路径是否存在,如果不存在则创建
    output_path = f"./tensorboard_logs/{tb_log_name}/"
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    tensorboard_log_path = f"./tensorboard_logs/{tb_log_name}/"

    if model_type == 'ppo':
        model = PPO("MlpPolicy", env, tensorboard_log=tensorboard_log_path)
    elif model_type == 'dqn':
        model = DQN("MlpPolicy", env, tensorboard_log=tensorboard_log_path)
    elif model_type == 'a2c':
        model = A2C("MlpPolicy", env, tensorboard_log=tensorboard_log_path)
    
    # 在开始训练之前生成GIF
    generate_gif(model, f"./tensorboard_logs/{tb_log_name}/before_training.gif")
    
    model.learn(total_timesteps=total_timesteps, tb_log_name=tb_log_name)
    
    # 在训练完成之后生成GIF
    generate_gif(model, f"./tensorboard_logs/{tb_log_name}/after_training.gif")
    
    return model


model_run1 = train_frozen_lake(testbed='Frozenlake', model='ppo', total_timesteps=60000, tb_log_name="Frozenlake-ppo-bugfree-600000")
# model_run2 = train_frozen_lake(testbed='Mountaincar', total_timesteps=10000, tb_log_name="Mountaincar_bug1")

Open terminal and use the following command to show tensorboard

In [ ]:
# tensorboard --logdir=./tensorboard_logs/